<a href="https://colab.research.google.com/github/ashioyajotham/Natural-Language-Processing/blob/main/Finetuning/Falcon/Fine_Tuning_Falcon_7B_for_Code_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
from datasets import load_dataset

# Specify the name of the dataset
#dataset_name = "yahma/alpaca-cleaned"
dataset_name = "tatsu-lab/alpaca"


# Load the dataset from the specified name and select the "train" split
dataset = load_dataset(dataset_name, split="train")

In [3]:
# We will be loading the Falcon 7B model, applying 4bit quantization to it, and then adding LoRA adapters to the model.
import torch

from transformers import FalconForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Defining the name of the Falcon model
model_name = "ybelkada/falcon-7b-sharded-bf16"

# Configuring the BitsAndBytes quantization
bnb_config = BitsAndBytesConfig(
load_in_4bit=True,
device_map = 'auto',
bnb_4bit_quant_type="nf4",
bnb_4bit_compute_dtype=torch.float16,
)

# Loading the Falcon model with quantization configuration
model = FalconForCausalLM.from_pretrained(
model_name,
quantization_config=bnb_config,
trust_remote_code=True
)

# Disabling cache usage in the model configuration
model.config.use_cache = False

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
You are using a model of type RefinedWebModel to instantiate a model of type falcon. This is not supported for all configurations of models and can yield errors.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [4]:
# Load the tokenizer for the Falcon 7B model with remote code trust
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# Set the padding token to be the same as the end-of-sequence token
tokenizer.pad_token = tokenizer.eos_token

In [5]:
# Import the necessary module for LoRA configuration
from peft import LoraConfig

# Define the parameters for LoRA configuration
lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

# Create the LoRA configuration object
peft_config = LoraConfig(
lora_alpha=lora_alpha,
lora_dropout=lora_dropout,
r=lora_r,
bias="none",
task_type="CAUSAL_LM",
target_modules=[
"query_key_value",
"dense",
"dense_h_to_4h",
"dense_4h_to_h",
]
)

In [6]:
from transformers import TrainingArguments
# Define the directory to save training results
output_dir = "./results"

# Set the batch size per device during training
per_device_train_batch_size = 1

# Number of steps to accumulate gradients before updating the model
gradient_accumulation_steps = 4

# Choose the optimizer type (e.g., "paged_adamw_32bit")
optim = "paged_adamw_32bit"

# Interval to save model checkpoints (every 10 steps)
save_steps = 10

# Interval to log training metrics (every 10 steps)
logging_steps = 10

# Learning rate for optimization
learning_rate = 2e-4

# Maximum gradient norm for gradient clipping
max_grad_norm = 0.3

# Maximum number of training steps
max_steps = 20

# Warmup ratio for learning rate scheduling
warmup_ratio = 0.03

# Type of learning rate scheduler (e.g., "constant")
lr_scheduler_type = "constant"

# Create a TrainingArguments object to configure the training process
training_arguments = TrainingArguments(
output_dir=output_dir,
per_device_train_batch_size=per_device_train_batch_size,
gradient_accumulation_steps=gradient_accumulation_steps,
optim=optim,
save_steps=save_steps,
logging_steps=logging_steps,
learning_rate=learning_rate,
fp16=True,  # Use mixed precision training (16-bit)
max_grad_norm=max_grad_norm,
max_steps=max_steps,
warmup_ratio=warmup_ratio,
group_by_length=True,
lr_scheduler_type=lr_scheduler_type,
)

In [7]:
dataset = dataset.map(lambda x: {"text": x["input"]+x["output"]})

# Import the SFTTrainer from the TRL library
from trl import SFTTrainer

# Set the maximum sequence length
max_seq_length = 512

# Create a trainer instance using SFTTrainer
trainer = SFTTrainer(
model=model,
train_dataset=dataset,
peft_config=peft_config,
dataset_text_field="text",
max_seq_length=max_seq_length,
tokenizer=tokenizer,
args=training_arguments,
)

In [8]:
# Iterate through the named modules of the trainer's model
for name, module in trainer.model.named_modules():

# Check if the name contains "norm"
  if "norm" in name:
	  # Convert the module to use torch.float32 data type
	  module = module.to(torch.float32)


trainer.train()

wandb: Currently logged in as: ashioyajotham. Use `wandb login --relogin` to force relogin


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
The current implementation of Falcon calls `torch.scaled_dot_product_attention` directly, this will be deprecated in the future in favor of the `BetterTransformer` API. Please install the latest optimum library with `pip install -U optimum` and call `model.to_bettertransformer()` to benefit from `torch.scaled_dot_product_attention` and future performance optimizations.


Step,Training Loss
10,1.691700
20,1.644300


TrainOutput(global_step=20, training_loss=1.6680020332336425, metrics={'train_runtime': 126.1853, 'train_samples_per_second': 0.634, 'train_steps_per_second': 0.158, 'total_flos': 367056368432640.0, 'train_loss': 1.6680020332336425, 'epoch': 0.0})

In [ ]:
# Save the model
trainer.save_model("./falcon-7b-code-gen")
#tokenizer.save_pretrained("./tokenizer")

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from huggingface_hub import whoami

whoami()
# you should see something like {'type': 'user',  'id': '...',  'name': 'Wauplin', ...}

In [ ]:
from huggingface_hub import create_repo

create_repo(repo_id="super-cool-model")


In [ ]:
trainer.push_to_hub("super-cool-model")

In [12]:
import transformers
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)
sequences = pipeline(
   "Generate a python script to create random numbers between 10 and 100",
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Result: Generate a python script to create random numbers between 10 and 100
This tutorial will teach you how to generate a random number between two numbers. It will generate a random number between 10 and 100 based on the user input. It will also show you how to create a python script to create random numbers between 10 and 100.
To generate a python script to create random numbers, you must first define two variables. The first variable will store the range of numbers between which the random numbers will be generated. The second variable will store the number of numbers that will be generated. You must first assign a random number between 10 and 100 to the range variable. Then use the range variable to create a random number between 10 and 100 and assign it to the second variable. Finally, create a python script to create random numbers between 10 and 100.
This script will take the two variables as input and use the values to generate random numbers between 10


In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda" # the device to load the model onto

#model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
#tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

messages = [
    {"role": "user", "content": "What is your favourite condiment?"},
    {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
    {"role": "user", "content": "Do you have mayonnaise recipes?"}
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)
#model.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


<|im_start|>user
What is your favourite condiment?<|im_end|>
<|im_start|>assistant
Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!<|im_end|>
<|im_start|>user
Do you have mayonnaise recipes?<|im_end|>
<|im_start|>assistant
Absolutely! Want me to share some? I can make you an amazing egg salad, or a tasty potato salad. I can even whip up some homemade mayo to go with your burgers!<|im_end|>

</div>/end snippet>

<script src="script.js"></script>

</body>
</html>
<|endoftext|>


In [33]:
from transformers import pipeline
generator = pipeline('text-generation', model = model, tokenizer=tokenizer)
generator("The Bible is", max_length = 100, do_sample = True, num_return_sequences=1)
## [{'generated_text': "Hello, I'm a language modeler. So while writing this, when I went out to meet my wife or come home she told me that my"},
##  {'generated_text': "Hello, I'm a language modeler. I write and maintain software in Python. I love to code, and that includes coding things that require writing"}, ...


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'The Bible is still a long way from it, and its most valuable and precious texts are now being replaced by a new generation of believers, and the best they can do is to continue building the Bible to help you with your lives and to learn from God Himself and His message in a new way.\n\nThe Best in the World was founded in 2003 and is designed to help you grow, strengthen, inspire, and expand life a bit. To celebrate the beginning of our new generation of believers:'}]